In [1]:
using LinearAlgebra
include("ncon.jl");

[5873.482917029676 4961.818712197332 5700.8286434847205 6023.467046922934 5687.083053488969 6090.270039107518 5978.1159471456485 6277.025694680517 5879.0500346349845 5988.104161161598; 5738.452082568745 4882.576669036246 5567.09864259022 5902.997453761836 5500.791896801787 5964.072347639627 5853.012610685473 6146.791653115772 5786.5839089704195 5804.4969217999005; 5889.444639232869 5035.494399204535 5758.228422357119 6139.896234194802 5705.3590468588845 6160.146107357789 6061.452786455212 6354.779654919001 5986.428192168169 6046.051114022753; 5772.492886937099 4943.190285334487 5659.25645195868 5997.883617362685 5586.603345641539 6047.346286790043 5948.913225541129 6239.035622724144 5906.765740855115 5900.738489057132; 5750.735863638605 4942.790009156972 5613.459798491351 5909.934067327802 5565.085043948465 5946.138873894088 5869.49353617358 6148.189894674795 5798.332684239515 5842.595525007426; 5868.6009953062685 5026.998220050783 5754.675318368121 6056.184485860674 5703.388980275885 

In [2]:
"""
doApplyHam(psiIn,hloc,N,usePBC)
------------------------
by Glen Evenbly (c) for www.tensors.net, (v1.2) - last modified 06/2020
------------------------
Applies local Hamiltonian (given as sum of nearest neighbor terms 'hloc')
to input state 'psiIn'. Number of lattice sites specified as 'N' while
'usePBC' determines whether open or periodic boundaries are used.
"""
function doApplyHam(psiIn,hloc,N,usePBC)

  d = size(hloc,1);
  psiOut = zeros(d^N,1);
  for k = 1:N-1
    # apply local Hamiltonian terms to sites [k,k+1]
    cont_inds = [[2],[2]];
    psi_temp = tensordot(reshape(hloc,d^2,d^2),
      reshape(psiIn,d^(k-1), d^2, d^(N-1-k)),cont_inds);
    psiOut = psiOut + reshape(permutedims(psi_temp,[2,1,3]),d^N);
  end

  if usePBC
    # apply periodic term
    cont_inds = [[3,4],[3,1]];
    psi_temp = tensordot(reshape(hloc,d,d,d,d),
      reshape(psiIn,d, d^(N-2), d),cont_inds);
    psiOut = psiOut + reshape(permutedims(psi_temp,[2,3,1]),d^N);
  end

  return psiOut
end

tensordot (generic function with 1 method)

In [ ]:
"""
tensordot(A,B,cont_inds):
Function for taking the produce of two tensors A and B, designed to mimic the
numpy tensordot. Input `cont_inds = [A_axes,B_axes]` where `A_axes` and
`B_axes` are vectors describing the indices to be contracted, e.g. set
cont_inds = [[2],[1]] to contract the 2nd index of A with the 1st index of B.
"""

"""LET US NOT CARE ABOUT HOW THE CODE WORKS INTERNALLY FOR A MOMENT.
LET US JUST CARE ABOUT WHAT THE CODE DOES WHICH IS CONTRACTING THE GIVEN INDICES OF TWO TENSORS"""

function tensordot(A, B, cont_inds)

  A_free = deleteat!(collect(1:ndims(A)), sort(cont_inds[1]))
  B_free = deleteat!(collect(1:ndims(B)), sort(cont_inds[2]))
  A_perm = vcat(A_free, cont_inds[1])
  B_perm = vcat(cont_inds[2], B_free)

  return reshape(
    reshape(
      permutedims(A, A_perm),
      prod(size(A)[A_free]),
      prod(size(A)[cont_inds[1]]),
    ) * reshape(
      permutedims(B, B_perm),
      prod(size(B)[cont_inds[2]]),
      prod(size(B)[B_free]),
    ),
    (size(A)[A_free]..., size(B)[B_free]...),
  )
end



In [5]:
"""
mainExactDiag.jl
---------------------------------------------------------------------
Script file for initializing exact diagonalization using the 'eigs' routine
(based on Arpack) for a 1D quantum system

by Glen Evenbly (c) for www.tensors.net, (v1.2) - last modified 06/2020
"""

using Printf, LinearAlgebra, LinearMaps, Arpack

#### Your paths go here
include("ncon.jl");
include("doApplyHam.jl");

##### Simulation parameters
model = "XX" # select 'XX' or 'ising' models
Nsites = 18; # number of lattice sites
usePBC = true; # use periodic or open boundaries
numval = 1; # number of eigenstates to compute

##### Define Hamiltonian (quantum XX model)
d = 2; # local dimension
sX = [0 1; 1 0]; sY = [0 -im; im 0]; sZ = [1 0; 0 -1]; sI = [1 0; 0 1];
if model == "XX" # (THERE IS NO Z COMPONENT ACTING ON ANYTHING)
    hloc = reshape(real(kron(sX,sX) + kron(sY,sY)),2,2,2,2);
    EnExact = -4/sin(pi/Nsites); # for PBC
elseif model == "ising" # NOT CLEAR HOW THIS IS AN ISING MODEL
    hloc = reshape(-kron(sX,sX) + 0.5*kron(sI,sZ) + 0.5*kron(sZ,sI),2,2,2,2);
    EnExact = -2/sin(pi/(2*Nsites)); # for PBC
end
;

[6368.382870550811 6817.949360110565 6695.8668450740415 6519.985703597504 6368.783576290623 6033.214228895046 6358.516845744462 6054.125712296396 6336.371007705049 7209.672245225955; 6425.534659130287 6811.278669574932 6721.635194639772 6533.051533263355 6437.058945205049 6072.156945054065 6389.783132027015 6042.136681555942 6329.60791572369 7239.280787322804; 6384.156433218923 6838.805731227697 6715.175388731897 6516.4537117502205 6440.296634029251 6078.0484676458545 6403.178063852508 6081.606622189206 6321.158043169292 7220.382500409666; 6561.460247718046 7023.726129093094 6893.816266684133 6678.03489579221 6589.950120334517 6245.741451988153 6547.993446894468 6264.004573286804 6499.573505600168 7420.5271339839355; 6456.43287853153 6877.6567234080285 6753.451973513303 6573.213723491141 6484.562061340033 6087.1605245848705 6453.638963094575 6101.12542136921 6386.445331701938 7319.622225416912; 6379.742377909278 6810.904378694084 6688.060669192163 6473.394621219767 6407.283414553216 60

In [10]:
##### Recast the `doApplyHam` function as a LinearMap
c = doApplyHam(psiIn,hloc,Nsites,usePBC);
doApplyHamClosed = LinearMap(psiIn -> doApplyHam(psiIn,hloc,Nsites,usePBC), d^Nsites;
  ismutating=false, issymmetric=true, ishermitian=true, isposdef=false)

LoadError: UndefVarError: psiIn not defined

In [9]:
size(doApplyHamClosed)

((262144, 262144), 262144)

In [ ]:

##### Diagonalize Hamiltonian with eigs
diagtime = @elapsed Energy, psi = eigs(doApplyHamClosed; nev = numval,
  tol = 1e-12, which=:SR, maxiter = 300);

##### Check with exact energy
EnErr = Energy[1] - EnExact; # should equal to zero
@printf "NumSites: %d, Time: %d, Energy: %e, EnErr: %e \n" Nsites diagtime Energy[1] EnErr

In [4]:
# Checking how the tensordot function works
A = rand(d,d,d,d); B = rand(d,d,d);

In [6]:
tnes_dot = tensordot(A,B,[[1],[2]]);

In [7]:
size(tnes_dot)

(10, 10, 10, 10, 10)

In [21]:
size([1,2,3,4,6,9,8,7], 1)

8

In [14]:
size([1,2,3,4])

(4,)

In [23]:
(size([2 4; 6 8; 10 12], 1)) 

3